In [7]:
import pandas as pd

# Load the CSV file
data = pd.read_csv('../Baseball_ML/data/kml/college_raw.csv')

# Display the first few rows of the dataframe
data.head()

from shapely.geometry import Polygon
import geopandas as gpd
import numpy as np

def parse_coordinates(coordinate_string):
    coordinates = coordinate_string.split()
    coordinates = [tuple(map(float, coord.split(','))) for coord in coordinates]
    return coordinates

def create_polygon(coordinate_string):
    coordinates = parse_coordinates(coordinate_string)
    return Polygon(coordinates)

# Create a new DataFrame
geo_df = data.copy()

# Extract home plate coordinates (first/last point of 'fop' polygon)
geo_df['home_plate'] = geo_df['fop'].apply(lambda x: parse_coordinates(x)[0])

# Create Polygon objects for 'fop'
geo_df['fop'] = geo_df['fop'].apply(create_polygon)

# Convert DataFrame to GeoDataFrame
geo_df = gpd.GeoDataFrame(geo_df, geometry='fop')

# Display the first few rows
geo_df.head()


,field,foul,fop,home_plate
0,Adrian College,"-84.0697145,41.901861,0 -84.0703958,41.9026485...","POLYGON Z ((-84.06971 41.90186 0.00000, -84.06...","(-84.0697145, 41.901861, 0.0)"
1,Alabama - Sewell–Thomas Stadium,"-87.5387466,33.2052609,0 -87.53780449999999,33...","POLYGON Z ((-87.53875 33.20526 0.00000, -87.53...","(-87.5387466, 33.2052609, 0.0)"
2,Albion College - college,"-84.7373745,42.241567,0 -84.7373704,42.2424775...","POLYGON Z ((-84.73737 42.24157 0.00000, -84.73...","(-84.7373745, 42.241567, 0.0)"
3,Arizona - Hi Corbett Field - college,"-110.9194283,32.2131913,0 -110.9194169,32.2121...","POLYGON Z ((-110.91943 32.21319 0.00000, -110....","(-110.9194283, 32.2131913, 0.0)"
4,Arkansas - Baum-Walker Stadium - college,"-94.1827419,36.0502882,0 -94.1816529,36.050265...","POLYGON Z ((-94.18274 36.05029 0.00000, -94.18...","(-94.1827419, 36.0502882, 0.0)"


In [9]:
from shapely.geometry import Point
import math

def calculate_angle(point, home_plate):
    # Convert to radians
    point_rad = (math.radians(point[0]), math.radians(point[1]))
    home_plate_rad = (math.radians(home_plate[0]), math.radians(home_plate[1]))

    # Calculate difference in coordinates
    diff_longitude = point_rad[0] - home_plate_rad[0]
    diff_latitude = point_rad[1] - home_plate_rad[1]

    # Calculate angle
    angle = math.atan2(diff_latitude, diff_longitude)
    
    # Convert angle to degrees and adjust range to 0-360
    angle = math.degrees(angle)
    if angle < 0:
        angle += 360

    return angle

def divide_field(polygon, home_plate):
    # Get coordinates of the polygon
    coordinates = list(polygon.exterior.coords)
    
    # Calculate angle for each point and assign to segment
    segments = {"down_the_line_left": [], "left_porch": [], "left_power_alley": [], 
                "center_field": [], "right_power_alley": [], "right_porch": [], 
                "down_the_line_right": []}
    for point in coordinates:
        angle = calculate_angle(point, home_plate)
        
        # Assign point to segment based on angle
        if 0 <= angle < 5 or 355 <= angle <= 360:
            segments["down_the_line_left"].append(point)
        elif 5 <= angle < 15:
            segments["left_porch"].append(point)
        elif 15 <= angle < 20:
            segments["left_power_alley"].append(point)
        elif 20 <= angle < 160:
            segments["center_field"].append(point)
        elif 160 <= angle < 165:
            segments["right_power_alley"].append(point)
        elif 165 <= angle < 175:
            segments["right_porch"].append(point)
        elif 175 <= angle < 180 or 180 <= angle < 355:
            segments["down_the_line_right"].append(point)
            
    # Convert point lists to Polygons only if they have enough points
    for key in segments:
        if len(segments[key]) >= 3:
            segments[key] = Polygon(segments[key])
        else:
            segments[key] = None

    return segments

# Calculate field segments
geo_df['segments'] = geo_df.apply(lambda x: divide_field(x['fop'], x['home_plate']), axis=1)

# Display the first row of the DataFrame
geo_df.head(20)


,field,foul,fop,home_plate,segments
0,Adrian College,"-84.0697145,41.901861,0 -84.0703958,41.9026485...","POLYGON Z ((-84.06971 41.90186 0.00000, -84.06...","(-84.0697145, 41.901861, 0.0)","{'down_the_line_left': None, 'left_porch': Non..."
1,Alabama - Sewell–Thomas Stadium,"-87.5387466,33.2052609,0 -87.53780449999999,33...","POLYGON Z ((-87.53875 33.20526 0.00000, -87.53...","(-87.5387466, 33.2052609, 0.0)","{'down_the_line_left': None, 'left_porch': Non..."
2,Albion College - college,"-84.7373745,42.241567,0 -84.7373704,42.2424775...","POLYGON Z ((-84.73737 42.24157 0.00000, -84.73...","(-84.7373745, 42.241567, 0.0)",{'down_the_line_left': POLYGON Z ((-84.7373745...
3,Arizona - Hi Corbett Field - college,"-110.9194283,32.2131913,0 -110.9194169,32.2121...","POLYGON Z ((-110.91943 32.21319 0.00000, -110....","(-110.9194283, 32.2131913, 0.0)","{'down_the_line_left': None, 'left_porch': Non..."
4,Arkansas - Baum-Walker Stadium - college,"-94.1827419,36.0502882,0 -94.1816529,36.050265...","POLYGON Z ((-94.18274 36.05029 0.00000, -94.18...","(-94.1827419, 36.0502882, 0.0)",{'down_the_line_left': POLYGON Z ((-94.1827419...
5,Auburn - Plainsman Park,"-85.4899848,32.6000522,0 -85.4889635,32.600013...","POLYGON Z ((-85.48998 32.60005 0.00000, -85.49...","(-85.4899848, 32.6000522, 0.0)",{'down_the_line_left': POLYGON Z ((-85.4899848...
6,Bailey-Brayton Field - Washington State - college,"-117.1553757920268,46.73449929788315,0 -117.15...","POLYGON Z ((-117.15435 46.73510 0.00000, -117....","(-117.1543531753322, 46.73509599790806, 0.0)","{'down_the_line_left': None, 'left_porch': Non..."
7,Barcroft Park - George Washington - college,"-77.10360206252069,38.85133095123716,0 -77.102...","POLYGON Z ((-77.10444 38.85070 0.00000, -77.10...","(-77.10444390524368, 38.85070498445064, 0.0)",{'down_the_line_left': POLYGON Z ((-77.1044439...
8,Baylor Ballpark - Baylor University - college,"-97.10748390625123,31.55076387358279,0 -97.108...","POLYGON Z ((-97.10697 31.55155 0.00000, -97.10...","(-97.10696528644037, 31.55155418276967, 0.0)","{'down_the_line_left': None, 'left_porch': Non..."
9,Bill Beck Field - Rhode Island - college,"-71.5387131102496,41.48508593502842,0 -71.5387...","POLYGON Z ((-71.53752 41.48497 0.00000, -71.53...","(-71.5375174187106, 41.48496753077652, 0.0)","{'down_the_line_left': None, 'left_porch': Non..."
